In [1]:
import sys
sys.path.append("../")
import os
data_path = "./LiuZhongjing-All-In"
if not os.path.isdir(data_path):
    os.system("git clone https://github.com/LiuZhongjing/LiuZhongjing-All-In.git")

In [26]:
import json
import requests
import re
import numpy as np
import pickle
import gzip
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI # API v1.2

with open("../openai_key.txt", "r") as f:
    openai_key = f.read().strip()
    openai_client = OpenAI(api_key=openai_key)

In [12]:
def get_all_files_in_folder(folder_path):
    file_list = []
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            if filename=="README.md": continue
            if not filename.endswith(".md"): continue
            file_list.append(os.path.join(root, filename))
    return file_list

file_list = get_all_files_in_folder(data_path)
file_list = sorted(file_list)
print(len(file_list))

101


In [29]:
def extract_titles(files, out_file="titles.json"):
    """提取每个文档的标题，并按照编号整理到一个json文件中"""
    dict1 = dict()
    for id, in_file in enumerate(tqdm(files)):
        file_name = in_file.split("/")[-1].split(".")[0]
        with open(in_file, "r") as f:
            title = f.readlines()[0]
        dict1[file_name] = title
    with open(out_file, "w") as f:
        json.dump(dict1, f)

def load_titles(file="titles.json"):
    """读取标题文件"""
    with open(file, "r") as f:
        dict1 = json.load(f)
    return dict1

extract_titles(file_list, out_file="titles.json")
dict1 = load_titles(file="titles.json")
#print(dict1)

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [00:00<00:00, 233.02it/s]


In [17]:
def load_data_to_paragraphs(file):
    """ 将长文档分解成1000字以内短文档. 
    因为openai sentence embedding ada 002 8000 input token, 最大2000汉字
    但是逼近2000后语义编码效果会下降
    """
    with open(file, "r") as f:
        data = f.read()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    txts = text_splitter.split_text(data)
    return txts

def sentence_embedding_batch(txts, file_name):
    """将list of text编码为sentence embedding 1536维"""
    l1 = []
    #embs = get_embeddings(txts, engine="text-embedding-ada-002") # old api
    response = openai_client.embeddings.create(input = txts, model="text-embedding-ada-002")
    response = json.loads(response.json())["data"]
    embs = [x["embedding"] for x in response]
    for i, txt in enumerate(txts):
        label = f"{file_name}-{i}"
        emb = embs[i]
        l1.append((label, txt, emb))
    return l1

def encoding_file(in_file, output_dir):
    """将文档.txt文件分割并编码为sentence embedding，压缩保存为同名.npz文件"""
    file_name = os.path.basename(in_file).split(".")[0]
    out_file = os.path.join(output_dir, id+".npz")
    txts = load_data_to_paragraphs(in_file)
    packs = sentence_embedding_batch(txts, file_name)
    serialized_data = pickle.dumps(packs)
    compressed_data = gzip.compress(serialized_data)
    with open(out_file, "wb") as file:
        file.write(compressed_data)

def encoding_files(files, output_dir="./emb"):
    """批量将文件夹下txt文件编码为同名 embedding文件，2866个文件需要 openai 3美元"""
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    for i, in_file in enumerate(tqdm(files)):
        encoding_file(in_file, output_dir)

def decoding_file(file):
    with open(file, "rb") as f:
        compressed_data = f.read()
    decompressed_data = gzip.decompress(compressed_data)
    l1 = pickle.loads(decompressed_data)
    return l1

In [18]:
encoding_files(file_list, output_dir = "./emb") # 编码，保存到文件

100%|█████████████████████████████████████████████████████████████████████████████████| 101/101 [04:58<00:00,  2.96s/it]
